In [ ]:
import os
from astropy import table
from astropy.table import Table,vstack
from astropy.io import ascii
import astropy.constants
from astropy.coordinates import SkyCoord
import astropy.units as u
import SAGA
from easyquery import *
from SAGA import HostCuts as H

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
all_hosts = saga.host_catalog.load(query='good_hosts', include_stats=True)

In [ ]:
assert len(all_hosts) == 205
assert H.paper1_complete.count(all_hosts) == 8
assert H.paper2_complete.count(all_hosts) == 36
assert H.good.count(all_hosts) == 115

In [ ]:
has_ls = Query("COVERAGE_DECALS_DR6 >= 0.99") | "COVERAGE_DECALS_DR7 >= 0.99"
has_des = Query("COVERAGE_DES_DR1 >= 0.99")
has_sdss = Query("COVERAGE_SDSS >= 0.99")
has_any = (has_ls | has_des | has_sdss)

assert (has_any | (~H.good)).mask(all_hosts).all()

In [ ]:
# CONVERT RA/DEC TO RADIANS FOR PLOTTING
def convert_radec2radian(RA,Dec,origin=0):
 
    x = np.remainder(RA+360-origin,360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x           # reverse the scale: East to the left

    xrad = np.radians(x)
    yrad = np.radians(Dec)
    
    return xrad,yrad

In [ ]:
# DEFINE GALACTIC PLANE
glat = np.arange(0,360,1)*u.degree+218.*u.degree
glon = np.zeros(np.size(glat))*u.degree
c = SkyCoord(l=glat,b=glon,frame='galactic')
gplane = c.transform_to('icrs')  

In [ ]:
origin = 285.
fig=plt.figure(figsize=(17,10))
plt.rcParams.update({'font.size': 18})
plt.rcParams.update({'legend.fontsize': 18})

ax = fig.add_subplot(111, projection='mollweide')#,projection = 'hammer')
tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
tick_labels = np.remainder(tick_labels+360+origin,360)


def add_points(*queries, **plot_options):
    t = Query(*queries).filter(all_hosts)
    if "label" in plot_options:
        plot_options["label"] = plot_options["label"].format(len(t))
    xh, yh = convert_radec2radian(t['RA'], t['DEC'], origin=origin)
    ax.scatter(xh, yh, **plot_options)

# GOOD POTENTIAL HOSTS
add_points(~has_any, s=95, facecolors='none', edgecolors='k', label='Imaging not yet available')

# GOOD HOSTS W/IMAGING
add_points(H.good, has_sdss, s=95, facecolors='y', edgecolors='k', alpha=0.7, label='SDSS imaging')
add_points(H.good, has_ls,   s=95, facecolors='g', edgecolors='k', alpha=0.7, label='LS DR6/7 imaging')
add_points(H.good, has_des,  s=95, facecolors='b', edgecolors='k', alpha=0.5, label='DES DR1 imaging')

# COMPLETE HOSTS
add_points(H.paper2_complete, ~H.paper1_complete, s=140, marker='s', facecolors='r', edgecolors='k', alpha=0.99, label='Newly Complete Host ({})')
add_points(H.paper1_complete, s=140, marker='s', facecolors='orange', edgecolors='k', alpha=0.99, label='Paper I Complete ({})')

# PLOT GALAXY PLANE
xg,yg = convert_radec2radian(gplane.ra.degree,gplane.dec.degree,origin=origin)
ax.plot(xg,yg,'k',label='Galactic Plane')
ax.grid(True)

lg = ax.legend(loc="lower left", markerscale=1.05, bbox_to_anchor=(0.9, -0.03),title=r'$\bf{SAGA\,\,Milky\,\,Way\,\,Analogs}$', frameon=True)
ax.set_xticklabels(tick_labels.astype(int)) 
          
lg.get_title().set_position((-20, 0)) # -10 is a guess
#plt.tight_layout()
plt.savefig('/home/yymao/Downloads/figure_hosts.pdf', bbox_inches="tight")

plt.show()